In [1]:
import cv2
import numpy as np
import sys
sys.path.insert(0, 'D:/old_stuff/new_stuff/impro3000-code/branches/impro3000_1.0_beta/imaging_lib')
import color_deconvolution as cd
import ImageObject as im_o
from skimage.measure import regionprops
import scipy.ndimage as ndi
from keras.models import Model, load_model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Activation, UpSampling2D, BatchNormalization
from keras.optimizers import RMSprop, SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from os.path import join, isfile
from os import listdir
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.losses import sparse_categorical_crossentropy
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy
from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter

def make_tensor_one(mask_list):
    masks = (np.array(mask_list))
    out = np.zeros((masks.shape[0], masks.shape[1], masks.shape[2], 1))
    for i in range(masks.shape[0]):
            sub_mat = masks[i,:,:]
            if 8 in sub_mat:
                sub_mat = sub_mat * (sub_mat < 8)
            out[i,:,:,0] = sub_mat
    print(np.unique(out))
    #np.save('img_masks.npy', out)
    return  out

def get_transformation_matrix(image, alpha, sigma, alpha_affine, random_state=None):
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    shape_size = shape[:2]
    
    # Random affine
    center_square = np.float32(shape_size) // 2
    square_size = min(shape_size) // 3
    pts1 = np.float32([center_square + square_size, [center_square[0]+square_size, center_square[1]-square_size], center_square - square_size])
    pts2 = pts1 + random_state.uniform(-alpha_affine, alpha_affine, size=pts1.shape).astype(np.float32)
    M = cv2.getAffineTransform(pts1, pts2)
    return M


def transform_channel(image, M, alpha, sigma, random_state=None):
    shape = image.shape
    shape_size = shape[:2]
    image = cv2.warpAffine(image, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)
    shape = image.shape
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
    #dz = np.zeros_like(dx)
    #x, y, z = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]), np.arange(shape[2]))
    #indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))
    x, y = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]))
    indices = np.reshape(y+dy, (-1,1)), np.reshape(x+dx, (-1,1))
    return map_coordinates(image, indices, order=1, mode='reflect').reshape(shape)

def elastic_transform(image, mask, alpha, sigma, alpha_affine, random_state=None):
    """
    Taken from: https://www.kaggle.com/bguberfain/elastic-transform-for-data-augmentation
    
    Elastic deformation of images as described in [Simard2003]_ (with modifications).
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
         Convolutional Neural Networks applied to Visual Document Analysis", in
         Proc. of the International Conference on Document Analysis and
         Recognition, 2003.

     Based on https://gist.github.com/erniejunior/601cdf56d2b424757de5
    """
    hem_image = image[:,:,0]
    cd_image = image[:,:,2]
    if random_state is None:
        random_state = np.random.RandomState(None)
    M = get_transformation_matrix(hem_image, alpha, sigma, alpha_affine, random_state=random_state)
    hem_trans = transform_channel(hem_image, M, alpha, sigma, random_state=random_state)
    cd_trans = transform_channel(cd_image, M, alpha, sigma, random_state=random_state)
    mask_trans = transform_channel(mask,  M, alpha, sigma, random_state=random_state)
    out_im = np.zeros(image.shape)
    out_im[:,:,0] = hem_trans
    out_im[:,:,2] = cd_trans
    return out_im, mask_trans
    
    

def deform_image(image, mask, state=None):
    width = image.shape[0]
    alpha = width * 2
    sigma = float(width) * 0.08
    alpha_affine = float(width) * 0.08
    out_image, out_mask = elastic_transform(image, mask, alpha, sigma, alpha_affine, random_state=state)
    return out_image, out_mask

def augment_simple(ims, masks, angle_range=90, width_shift_range=30, height_shift_range=30, do_deformation=False):
    out_ims = []
    out_masks = []
    for i in range(len(ims)):
        im = ims[i]
        mask = masks[i]
        height, width = im.shape[:2]
        #mask = masks[i]
        ### adding un-augmented images to training data
        ##out_ims.append(im)
        if (np.sum(mask) != 0):
            out_ims.append(im)
            out_masks.append(mask)
            elems = np.unique(mask)
            if ((1 in elems) or (2 in elems) or (3 in elems)):
                Mrot2 = cv2.getRotationMatrix2D((width / 2, height / 2), int(np.random.uniform(low=0, high=angle_range)), 1)
                aug_im3 = cv2.warpAffine(im, Mrot2, (width, height))
                aug_mask3 = cv2.warpAffine(mask, Mrot2, (width, height))
                #M = np.float32([[1,0,np.random.uniform(low=0.0, high=width_shift_range)],[0,1,np.random.uniform(low=0.0, high=height_shift_range)]])  ### matrix for shifting
                #aug_im_h = cv2.warpAffine(im, M, (width, height))
                #aug_mask_h = cv2.warpAffine(mask, M, (width, height))
                out_ims.append(aug_im3)
                out_masks.append(aug_mask3)
                #out_ims.append(aug_im_h)
                #out_masks.append(aug_mask_h)
                if do_deformation:
                    im_def, mask_def = deform_image(im, mask)
                    out_ims.append(im_def)
                    out_masks.append(mask_def)
    return out_ims, out_masks

def get_train_data(im_folder, mask_folder):
    paths_ims = [join(im_folder, f) for f in listdir(im_folder) if isfile(join(im_folder, f))]
    paths_masks = [join(mask_folder, f) for f in listdir(mask_folder) if isfile(join(mask_folder, f))]
    ims = [(cv2.imread(f).astype(np.float64) / 255.) for f in paths_ims]
    ims = [cv2.resize(f, (0,0), fx=0.5, fy=0.5) for f in ims]
    masks = [cv2.imread(f, 0) for f in paths_masks]
    masks = [cv2.resize(f, (0,0), fx=0.5, fy=0.5) for f in masks]
    class_hist = np.zeros(8)
    for mask in masks:
        for i in range(8):
            class_hist[i] += np.sum(mask == i)
    print('class histogram is')
    print(class_hist * (1./np.sum(class_hist)))
    return ims, masks

def get_good_means(ims, ans):
    val_arrs = [[], [], [], [], [], [], [], []]
    means = np.zeros((8))
    for im, an in list(zip(ims, ans)):
        for i in range(1,9):
            mask = an == i
            vals = np.extract(mask, im[:,:,2])
            val_arrs[i-1] += list(vals)
    for i in range(8):
        if (len(val_arrs[i]) > 0):
            mean = np.mean(val_arrs[i])
            means[i] = mean
        else:
            means[i] = means[i-1] + 5.
    print('number of vals in each class is')
    for ll in val_arrs:
        print(len(ll))
    print('means are')
    print(means)
    return means

def transform_mask(im, mask, good_means):
    out = np.zeros(mask.shape)
    cell_im = (mask == 1) + (mask == 2)
    art_im = mask == 5
    uns_im = mask == 7
    lbl, num = ndi.label(cell_im)
    for i in range(1, num):
        m_val = np.mean(np.extract(lbl == i, im[:,:,2]))
        mean_diffs = np.array([np.linalg.norm(m_val-good_means[0]), np.linalg.norm(m_val-good_means[1]), np.linalg.norm(m_val-good_means[2])])
        new_label = np.argmin(mean_diffs) + 1
        out += ((lbl == i) * new_label)
    lbl, num = ndi.label(art_im)
    for i in range(1, num):
        m_val = np.mean(np.extract(lbl == i, im[:,:,2]))
        mean_diffs = np.array([np.linalg.norm(m_val-good_means[3]), np.linalg.norm(m_val-good_means[4]), np.linalg.norm(m_val-good_means[5])])
        new_label = np.argmin(mean_diffs) + 1
        out += ((lbl == i) * new_label)
    lbl, num = ndi.label(uns_im)
    for i in range(1, num):
        m_val = np.mean(np.extract(lbl == i, im[:,:,2]))
        mean_diffs = np.array([np.linalg.norm(m_val-good_means[6]), np.linalg.norm(m_val-good_means[7])])
        new_label = np.argmin(mean_diffs) + 1
        out += ((lbl == i) * new_label)
    return out
            

def make_training_images(folder_ims, folder_ans, out_folder_ims, out_folder_ans, index=0):
    im_names = [f for f in listdir(folder_ims) if ('.jpg' in f)]
    an_names = [f for f in listdir(folder_ans) if ('.tif' in f)]
    im_prefixes = []
    im_prefixes_good = []
    an_prefixes = []
    an_prefixes_good = []
    for im_name in im_names:
        if ('cut' in im_name):
            im_prefixes.append(im_name[:im_name.index('cut')])
            im_prefixes_good.append(im_name[:im_name.index('cut')])
        else:
            im_prefixes.append(im_name[:im_name.index('.jpg')])
    for an_name in an_names:
        if ('_multi' in an_name):
            an_prefixes.append(an_name[:an_name.index('_multi')])
            an_prefixes_good.append(an_name[:an_name.index('_multi')])
        elif ('_an' in an_name):
            an_prefixes.append(an_name[:an_name.index('_an')])
    ims = []
    ans = []
    good_ims = []
    good_ans = []
    print('Fetching data for good means')
    for im_pref in im_prefixes_good:
        im_index = im_prefixes.index(im_pref)
        an_index = an_prefixes.index(im_pref)
        im = cv2.imread(join(folder_ims, im_names[im_index]))
        im_object = im_o.ImageObject(numpy_array = cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
        cd_im, hem = cd.colour_deconvolution(im_object, path='D:/old_stuff/new_stuff/impro3000-code/branches/impro3000_1.0_beta/imaging_lib/color_deconvolution_lib/cd30_hematoxylin_rest.csv')
        im_seg = np.zeros(im.shape)
        im_seg[:,:,0] = hem.get_numpy_array()
        im_seg[:,:,2] = cd_im.get_numpy_array()
        im = im_seg
        an = cv2.imread(join(folder_ans, an_names[an_index]), 0)
        good_ims.append(im)
        good_ans.append(an)
    print('Got images for good means')
    good_means = get_good_means(good_ims, good_ans)
    index = 0
    
    for im_pref in im_prefixes:
        print('Processing ' + im_pref)
        im_index = im_prefixes.index(im_pref)
        an_index = an_prefixes.index(im_pref)
        im = cv2.imread(join(folder_ims, im_names[im_index]))
        im_object = im_o.ImageObject(numpy_array = cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
        cd_im, hem = cd.colour_deconvolution(im_object, path='D:/old_stuff/new_stuff/impro3000-code/branches/impro3000_1.0_beta/imaging_lib/color_deconvolution_lib/cd30_hematoxylin_rest.csv')
        im_seg = np.zeros(im.shape)
        im_seg[:,:,0] = hem.get_numpy_array()
        im_seg[:,:,2] = cd_im.get_numpy_array()
        im = im_seg
        an = cv2.imread(join(folder_ans, an_names[an_index]), 0)
        if (not (im_pref in im_prefixes_good)):
            print('Fixing image')
            an = transform_mask(im, an, good_means)
        print('Unique values are')
        print(np.unique(an))
        for i in range(0, im.shape[0], 256):
            for k in range(0, im.shape[1], 256):
                if (((i + 512) <= im.shape[0]) and ((k+512) <= im.shape[1])):
                    im_part = np.zeros((512,512,3))
                    im_part = np.copy(im[i:i+512,k:k+512])
                    an_part = np.zeros((512, 512))
                    an_part = np.copy(an[i:i+512,k:k+512])
                    im_name = str(index) + '.tif'
                    cv2.imwrite(join(out_folder_ims, im_name), im_part)
                    cv2.imwrite(join(out_folder_ans, im_name), an_part)
                    index += 1
    print('Finished writing all images')
    return

folder = 'D:/old_stuff/new_stuff/slightly_better_training'

def prepare_data(im_folder, mask_folder, im_width=256, do_augmentation=True):
    num_ims_wrong_shape = 0
    norm_params = np.loadtxt('im_mean_std_at3.txt')
    im_mean = norm_params[:3]
    im_std = norm_params[3]
    ims, masks = get_train_data(im_folder, mask_folder)
    #im_mean, im_std = #compute_mean_std(ims)
    out_arr = np.zeros(4)
    out_arr[:3] = im_mean
    out_arr[3] = im_std
    #np.savetxt('im_mean_std_at3', out_arr)
    print('loaded data')
    if do_augmentation:
        ims, masks = augment_simple(ims, masks, do_deformation=True)
    print('augmented data')
    class_hist = np.zeros(8)
    initial_shape = masks[0].shape
    print('initial shape is ')
    print(initial_shape)
    mask_index = 0
    ims_corrected = []
    ans_corrected = []
    for im, an in list(zip(ims, masks)):
        if (an.shape == initial_shape):
            ims_corrected.append(im)
            ans_corrected.append(an)
            for i in range(8):
                class_hist[i] += np.sum(an == i)
        else:
            num_ims_wrong_shape += 1
        mask_index += 1
    print('found {} images with wrong shapes'.format(num_ims_wrong_shape))
    print('class histogram after augmentation is')
    print(class_hist * (1./np.sum(class_hist)))
    ims = ims_corrected
    masks = ans_corrected
    sub_im = np.ones((im_width, im_width, 3), dtype=np.float32) * im_mean
    masks = make_tensor_one(masks)
    ims = [(im - sub_im) / im_std for im in ims]
    ims = np.array(ims)
    return ims, masks

#make_training_images(join(folder, 'big_images'), join(folder, 'big_annotations'), join(folder, 'images'), join(folder, 'annotations'))
#ims, masks = prepare_data(join(folder, 'images'), join(folder, 'annotations'))

Using TensorFlow backend.


In [2]:
from keras.optimizers import Adadelta
from sklearn.utils import class_weight
import random

def compute_weight_vector(masks):
    vector = np.zeros(8)
    for i in range(8):
        vector[i] = np.sum(masks == i)
    weights = float(masks.shape[0]) / (float(8) * vector)
    return weights

def make_train_test_split(ims, ans, split=0.3):
    split_index = int(float(len(ims)) * (1. - split))
    split_list = list(zip(ims, list(ans)))
    random.shuffle(split_list)
    train = split_list[:split_index]
    test = split_list[split_index:]
    train_ims, train_ans = zip(*train)
    test_ims, test_ans = zip(*test)
    return train_ims, np.array(train_ans), test_ims, np.array(test_ans)

print('starting data preparation')
ims, masks = prepare_data(join(folder, 'images'), join(folder, 'annotations'))
print('data prepared')

w_vec1 = compute_weight_vector(masks)
weight_vec_fin = w_vec1
for i in range(4,8):
    if w_vec1[i] >= np.amin(w_vec1[:4]):
        weight_vec_fin[i] = np.amin(w_vec1)
print(weight_vec_fin)

starting data preparation
class histogram is
[9.65233954e-01 2.64906097e-03 7.65291790e-03 2.31224154e-02
 5.25843374e-04 7.93934018e-04 9.58767770e-08 2.17784099e-05]
loaded data
augmented data
initial shape is 
(256, 256)
found 0 images with wrong shapes
class histogram after augmentation is
[9.46141715e-01 5.63586982e-03 1.18188724e-02 3.43689578e-02
 8.14366525e-04 1.19016166e-03 1.27219431e-06 2.87840248e-05]
[0. 1. 2. 3. 4. 5. 6. 7.]
data prepared
[2.01592278e-06 3.38430215e-04 1.61381607e-04 5.54962604e-05
 2.01592278e-06 2.01592278e-06 2.01592278e-06 2.01592278e-06]


In [ ]:
def load_validation_data(im_folder, mask_folder):
    paths_ims = [join(im_folder, f) for f in listdir(im_folder) if isfile(join(im_folder, f))]
    paths_masks = [join(mask_folder, f) for f in listdir(mask_folder) if isfile(join(mask_folder, f))]
    ims = [(cv2.imread(paths_ims[i]).astype(np.float64) / 255.) for i in range(0, len(paths_ims), 2)]
    ims = [cv2.resize(f, (0,0), fx=0.5, fy=0.5) for f in ims]
    masks = [cv2.imread(paths_masks[i], 0) for i in range(0, len(paths_masks), 2)]
    masks = [cv2.resize(f, (0,0), fx=0.5, fy=0.5) for f in masks]
    ims_corrected = []
    ans_corrected = []
    initial_shape = masks[0].shape
    im_width = initial_shape[0]
    print(initial_shape)
    for im, an in list(zip(ims, masks)):
        if (an.shape == initial_shape):
            ims_corrected.append(im)
            ans_corrected.append(an)
    norm_params = np.loadtxt('im_mean_std_at3.txt')
    im_mean = norm_params[:3]
    im_std = norm_params[3]
    ims = ims_corrected
    masks = ans_corrected
    sub_im = np.ones((im_width, im_width, 3), dtype=np.float32) * im_mean
    masks = make_tensor_one(masks)
    ims = [(im - sub_im) / im_std for im in ims]
    ims = np.array(ims)
    return ims, masks


def train_with_generator(ims, masks, weights, model_path='D:/old_stuff/new_stuff/impro3000-code/branches/impro3000_1.0_beta/imaging_lib/u-net_e18.hdf5', num_epochs=10, start_epoch=0):
    """
    Should work
    """
    
    #ims, masks, test_ims, test_ans = make_train_test_split(ims, masks, split=0.2)
    #test_ims, test_ans = prepare_data(join(folder, 'validation_images'), join(folder, 'validation_annotations'), do_augmentation=False)
    test_ims, test_ans = load_validation_data(join(folder, 'validation_images'), join(folder, 'validation_annotations'))
    print(len(ims))
    print(masks.shape)
    print(len(test_ims))
    print(test_ans.shape)
    seed = 1
    train_generator = ImageDataGenerator()
    test_generator = ImageDataGenerator()
    train_generator.fit(ims, augment=False)
    test_generator.fit(test_ims, augment=False)
    if (start_epoch != 0):
        if (start_epoch < 10):
            model_path = 'D:/old_stuff/new_stuff/u-net_gen_newaug_e0{}.hdf5'.format(start_epoch)
        else:
            model_path = 'D:/old_stuff/new_stuff/u-net_gen_newaug_e{}.hdf5'.format(start_epoch)
    model = load_model(model_path)
    model.compile(optimizer=Adam(), loss=sparse_categorical_crossentropy, metrics=[categorical_accuracy])
    callbacks = [ModelCheckpoint(filepath='u-net_gen_newaug_e{epoch:02d}.hdf5',
                             period=1,
                             save_best_only=False,
                             save_weights_only=False)]
    #history = model.fit_generator(train_generator, validation_data=test_generator, callbacks=callbacks, epochs=num_epochs, verbose=1, class_weight=weights, steps_per_epoch=len(ims)*3, initial_epoch=start_epoch)
    if (not weights is None):
        history = model.fit_generator(train_generator.flow(ims, masks, batch_size=1), validation_data=test_generator.flow(test_ims, test_ans, batch_size=1), validation_steps=len(test_ims), callbacks=callbacks, epochs=num_epochs, verbose=1, class_weight=weights, steps_per_epoch=len(ims), initial_epoch=start_epoch)
    else:
        history = model.fit_generator(train_generator.flow(ims, masks, batch_size=1), validation_data=test_generator.flow(test_ims, test_ans, batch_size=1), validation_steps=len(test_ims), callbacks=callbacks, epochs=num_epochs, verbose=1, steps_per_epoch=len(ims), initial_epoch=start_epoch)
    np.savetxt('u-net_cat_acc_generator_s{}e{}.txt'.format(start_epoch, num_epochs), history.history['categorical_accuracy'])
    np.savetxt('u-net_val_cat_generator_s{}e{}.txt'.format(start_epoch, num_epochs), history.history['val_categorical_accuracy'])
    np.savetxt('u-net_loss_generator_s{}e{}.txt'.format(start_epoch, num_epochs), history.history['loss'])
    np.savetxt('u-net_val_loss_generator_s{}e{}.txt'.format(start_epoch, num_epochs), history.history['val_loss'])
    return


train_with_generator(ims, masks, None, num_epochs=25, start_epoch=6)

(256, 256)
[0. 1. 2. 3.]
12138
(12138, 256, 256, 1)
908
(908, 256, 256, 1)
Epoch 7/25
12138/12138 [==============================] - 3204s 264ms/step - loss: 0.1090 - categorical_accuracy: 0.9619 - val_loss: 0.0668 - val_categorical_accuracy: 0.9914
Epoch 8/25
12138/12138 [==============================] - 3191s 263ms/step - loss: 0.1055 - categorical_accuracy: 0.9611 - val_loss: 0.0674 - val_categorical_accuracy: 0.9917
Epoch 9/25
12138/12138 [==============================] - 3192s 263ms/step - loss: 0.1024 - categorical_accuracy: 0.9603 - val_loss: 0.0681 - val_categorical_accuracy: 0.9914
Epoch 10/25
12138/12138 [==============================] - 3191s 263ms/step - loss: 0.0998 - categorical_accuracy: 0.9597 - val_loss: 0.0745 - val_categorical_accuracy: 0.9947
Epoch 11/25
12138/12138 [==============================] - 3189s 263ms/step - loss: 0.0966 - categorical_accuracy: 0.9589 - val_loss: 0.0653 - val_categorical_accuracy: 0.9888
Epoch 12/25
12138/12138 [=======================